In [1]:
# imports
import torch
import numpy as np
from tools import load_experiment_models
from tools import uncertainty_plot

In [3]:
# Example usage:
device='cuda'
config_path = 'configs/test.conf'  # Path to the config file

# Load the models for the specified experiment
models_dict = load_experiment_models(config_path, device=device)

Configuration file configs/test.conf is found and has content.
Sections found in config: ['EXPERIMENT', 'TESTS']
Keys in section 'EXPERIMENT': ['batch_size', 'height', 'width', 'imgs', 'pth_file', 'experiment_name', 'base_directory', 'device', 'tn', 'tf', 'nb_epochs', 'lr', 'gamma', 'nb_bins', 'ensembles']
Keys in section 'TESTS': ['single_test', 'single_test_anomaly_imgs', 'max_anomaly_images', 'step_size', 'max_radius', 'radius_step']
current folder: set20
current folder: set30
current folder: set25
current folder: set0
current folder: set5
current folder: set15
current folder: set10
Loaded M0.pth successfully.
Loaded M0.pth successfully.
Loaded M0.pth successfully.
Loaded M0.pth successfully.
Loaded M0.pth successfully.
Loaded M0.pth successfully.
Loaded M0.pth successfully.


In [6]:
# extracting the densities
N = 100
scale = 1.5

x = torch.linspace(-scale, scale, N)
y = torch.linspace(-scale, scale, N)
z = torch.linspace(-scale, scale, N)

x, y, z = torch.meshgrid((x, y, z))

xyz = torch.cat((x.reshape(-1, 1),
                 y.reshape(-1, 1),
                 z.reshape(-1, 1)), dim=1)

/home/eherrin@ad.ufl.edu/anaconda3/envs/nerf3Dchange/lib/python3.12/site-packages/torch/functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3587.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [7]:
# arithmetic mean of each point (x,y,z) in our 3D grid
densities = []
for model in models_dict['set0']:

    with torch.no_grad():
        _, density = model.forward(xyz.to(device), torch.zeros_like(xyz).to(device))
    
    #density = density.cpu().numpy().reshape(N, N, N)
    density = density.view(N, N, N)
    densities.append(density)

densities_tensor = torch.stack(densities)
mean_density = torch.mean(densities_tensor, dim=0) # calculates the arithmetic mean for every point

In [8]:
# for all densities across the 7 models, substract the model density from the mean
squared_diffs = []

for model_density in densities: # for each model, subtact the mean density from each element.

    squared_diffs.append(torch.square(torch.sub(model_density, mean_density))) # squared diffference for all points

tensor_sum = torch.sum(torch.stack(squared_diffs), dim=0)
divisor = len(models_dict['set0']) - 1
sample_variance = torch.div(tensor_sum, divisor)
coeff_variation = torch.div(torch.sqrt(sample_variance), mean_density)

In [10]:
coeff_variation = coeff_variation.cpu().numpy().reshape(N, N, N)
sample_variance = sample_variance.cpu().numpy().reshape(N, N, N)
sf_mean_density = mean_density.cpu().numpy().reshape(N, N, N)

AttributeError: 'numpy.ndarray' object has no attribute 'cpu'

In [11]:
pts, values = uncertainty_plot(scalar_field = sf_mean_density, scalars=None, pts=None, threshold=10)

/home/eherrin@ad.ufl.edu/anaconda3/envs/nerf3Dchange/lib/python3.12/site-packages/pyvista/utilities/helpers.py:507: UserWarning: Points is not a float type. This can cause issues when transforming or applying filters. Casting to ``np.float32``. Disable this by passing ``force_float=False``.
  warnings.warn(


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [12]:
from scipy.spatial import KDTree

# Note: neighbor idx returns the index in the array of all pts of the neighbors.
map_tree = KDTree(pts)
neighbor_idx = map_tree.query_ball_point(x=pts, r=4) # returns list of indices of neighbors of x

In [16]:
CoV = [] # list of coefficent of variation for each point based on variation of neighbor densities
for i in range(len(neighbor_idx)):
    neighbors = neighbor_idx[i] # [len(neighbors of that point)]

    surrounding_densities = [] # reinit list of neighbor for each point in the nerf
    for j in range(len(neighbors)):
        surrounding_densities.append(values[neighbors[j]]) # [n] densitity at each index in neighbors

    densities_tensor = torch.Tensor(surrounding_densities)
    mean_density = torch.mean(densities_tensor, dim=0) # calculates the arithmetic mean for every point

    squared_diffs = []
    for d in densities_tensor: # for each point, subtact the mean density from each element.

        squared_diffs.append(torch.square(torch.sub(d, mean_density))) # squared diffference for all points
    
    tensor_sum = torch.sum(torch.stack(squared_diffs), dim=0)
    divisor = len(models_dict['set0']) - .9999
    sample_variance = torch.div(tensor_sum, divisor)
    coeff_variation = torch.div(torch.sqrt(sample_variance), mean_density)
    CoV.append(coeff_variation)
    

In [17]:
uncertainty_plot(scalar_field=None, scalars=np.array(CoV), pts=pts, threshold=3.5)

/home/eherrin@ad.ufl.edu/anaconda3/envs/nerf3Dchange/lib/python3.12/site-packages/pyvista/utilities/helpers.py:507: UserWarning: Points is not a float type. This can cause issues when transforming or applying filters. Casting to ``np.float32``. Disable this by passing ``force_float=False``.
  warnings.warn(


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

0

In [20]:
object_mask = np.load('monkey_mask.npy') # [3,len(points)]
x_coords, y_coords, z_coords = object_mask
mask_pts = np.stack((x_coords, y_coords, z_coords), axis=-1)

In [21]:
uncertainty_plot(scalar_field=None, scalars=np.array(CoV), pts=mask_pts, threshold=3.5)

IndexError: index 6967 is out of bounds for axis 0 with size 6967